<a href="https://colab.research.google.com/github/liamlio/tensorflowfun/blob/master/CalHousing_Regression_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Playing around with the Sequential API and Non-Sequential API of Keras using the california housing dataset

In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

TensorFlow 2.x selected.


In [0]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)



In [5]:
model = keras.models.Sequential([
      keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
      keras.layers.Dense(1) #Only looking for one output
])

model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 2s 138us/sample - loss: 0.8082 - val_loss: 0.8208
Epoch 2/20
11610/11610 [==============================] - 1s 57us/sample - loss: 1.1059 - val_loss: 0.5305
Epoch 3/20
11610/11610 [==============================] - 1s 60us/sample - loss: 0.5094 - val_loss: 0.4508
Epoch 4/20
11610/11610 [==============================] - 1s 63us/sample - loss: 0.4591 - val_loss: 0.4162
Epoch 5/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.4338 - val_loss: 0.3994
Epoch 6/20
11610/11610 [==============================] - 1s 59us/sample - loss: 0.4176 - val_loss: 0.3883
Epoch 7/20
11610/11610 [==============================] - 1s 54us/sample - loss: 0.4099 - val_loss: 0.3780
Epoch 8/20
11610/11610 [==============================] - 1s 56us/sample - loss: 0.4008 - val_loss: 0.3783
Epoch 9/20
11610/11610 [==============================] - 1s 54us/sample - loss: 0.3919 - val_

In [0]:
#Let make a Wide&Deep network
# Essentially just a nonsequential model where simple patterns don't have to go through the
#the deep aspect of our neural network(The hidden layers)
#this is dont through Keras' functional API
input_ = keras.layers.Input(shape=X_train.shape[1:]) #input object
hidden1 = keras.layers.Dense(30, activation="relu")(input_)#We pass the preceding layer like a function
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2]) #concatenate the input and output of the second layer
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output]) #Creating the Keras Model

In [0]:
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

In [0]:
#Now to actually create a wide and deep nn
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [26]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 87us/sample - loss: 2.2427 - val_loss: 1.3920
Epoch 2/20
11610/11610 [==============================] - 1s 60us/sample - loss: 0.8678 - val_loss: 0.8406
Epoch 3/20
11610/11610 [==============================] - 1s 67us/sample - loss: 0.7351 - val_loss: 0.6914
Epoch 4/20
11610/11610 [==============================] - 1s 69us/sample - loss: 0.6755 - val_loss: 0.6216
Epoch 5/20
11610/11610 [==============================] - 1s 73us/sample - loss: 0.6358 - val_loss: 0.5835
Epoch 6/20
11610/11610 [==============================] - 1s 65us/sample - loss: 0.6055 - val_loss: 0.5540
Epoch 7/20
11610/11610 [==============================] - 1s 70us/sample - loss: 0.5811 - val_loss: 0.5323
Epoch 8/20
11610/11610 [==============================] - 1s 71us/sample - loss: 0.5612 - val_loss: 0.5149
Epoch 9/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.5447 - val_l

In [0]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [0]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1],
              optimizer="sgd")
#We care more about the first output since the aux output is used for regurlarization

In [30]:
history = model.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs=20,
    validation_data=([X_valid_A, X_valid_B, [y_valid, y_valid]))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 2s 135us/sample - loss: 1.1306 - main_output_loss: 1.0497 - aux_output_loss: 1.8545 - val_loss: 1.3749 - val_main_output_loss: 1.3467 - val_aux_output_loss: 1.6246
Epoch 2/20
11610/11610 [==============================] - 1s 68us/sample - loss: 0.6138 - main_output_loss: 0.5586 - aux_output_loss: 1.1102 - val_loss: 0.5465 - val_main_output_loss: 0.5025 - val_aux_output_loss: 0.9423
Epoch 3/20
11610/11610 [==============================] - 1s 86us/sample - loss: 0.5206 - main_output_loss: 0.4801 - aux_output_loss: 0.8836 - val_loss: 0.4490 - val_main_output_loss: 0.4126 - val_aux_output_loss: 0.7762
Epoch 4/20
11610/11610 [==============================] - 1s 80us/sample - loss: 0.4735 - main_output_loss: 0.4435 - aux_output_loss: 0.7433 - val_loss: 0.4483 - val_main_output_loss: 0.4055 - val_aux_output_loss: 0.8334
Epoch 5/20
11610/11610 [==============================] - 1s 81us/

In [32]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])

5160/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================